# 지난시간에는...

### 결정트리 알고리즘
지니불순도가 0이 되었다는 것 = 리프노드 = 퓨어노드(순수노드) = 끝까지 갔다는 뜻 = Overfitting!
지니불순도에 따라서 어떻게 트리가 나눠지는지 복습해보아야겠다.....

### 검증세트
원래는 훈련세트 80%, 테스트 세트 20%로 나누어 테스트 세트를 통해 하이퍼파라메타를 결정해주었다.  
하지만 이렇게 하면 테스트 세트에 오버피팅이 될 수도 있기 때문에 그것에 대한 해결방법으로  
훈련세트 80%를 60%, 20%로 나누어, 훈련세트와 "검증세트"를 추가로 만들었었다.  
학습을 시키게 될 때, 훈련+검증 세트로 훈련을 시키고, score를 내게 될 때 검증 세트도 같이 score를 내서  
테스트 세트에 오버피팅 되지 않도록 만들어준다.

### 교차 검증
훈련세트, 검증세트를 데이터 안에서 K번 섞어서(?) 모델을 훈련하고 모델을 평가함으로써 오버피팅을 막는 방법이다.  
K번 섞는다고 해서 ***K-폴드 교차검증*** 이라고 한다.  
사이킷런에서는 5폴드가 기본으로 제공된다.  
함수 이름은 `cross_validate(model, train_input, train_target)`

# 이번 시간에는...

### 머신러닝
- 지도학습(supervised learning) - feature(X) + class(Y)
  - 회귀(예측) - Y값이 연속적
  - 분류 - Y값이 이산적
  - **지도학습의 한계** : Y값을 누군가가 정해주어야하기 때문에 인력이 필요하다. = 비용이 발생한다.
- 비지도학습(self-supervised learning) - feature(X)만 있는 것
  - 클러스터링(군집화) : 유사한 것 끼리 모아준다.
- 강화학습(reinforced learning) - X값 마저 기계가 찾는다
  - 상벌제도(Reward)

### 정형 vs 비정형 데이터
정형 데이터(Structured)는 숫자로 나타나는 데이터고,  
비정형 데이터(Unstructured)는 숫자로 나타나지 않는 데이터다.(텍스트, 오디오, 이미지, 영상...)  
보통은 정형 데이터는 머신러닝으로 처리하고, 비정형 데이터는 딥러닝으로 처리한다.

## 트리 앙상블 알고리즘
지금까지 배운 머신러닝 알고리즘 중에 가장 성능이 뛰어난 알고리즘  

트리 앙상블에는 두가지 방법이 있는데  
하나는 Bagging(병렬적),  
하나는 Boosting(직렬적)

대표적인 트리 앙상블 알고리즘은 ***랜덤 포레스트***  
코드로는 `RandomForestClassifier` & `RandomForestRegressor`  
결정 트리를 랜덤하게 만들어서 숲을 만드는 알고리즘이다.

### 랜덤 포레스트
#### 부트스트랩 샘플
첫번째, 랜덤 포레스트는 ***부트스트랩 샘플*** 이라는 것을 뽑아낸다.  
**부트스트랩 샘플 :** 중복을 허용하여 원래 데이터 크기 만큼 랜덤하게 뽑아낸다.  

>예)  
**동그라미1, 네모2, 세모2** 이렇게 있는데  
부트스트랩 샘플을 뽑아내면  
**세모3, 동그라미2** 이렇게 뽑아낼 수 있다는 말

두번째, 데이터셋에는 여러가지 특성들의 집합이 있는데,  
랜덤포레스트는 여기서 특성을 $\sqrt{특성개수}$ 만큼 랜덤으로 선택한다.  
특성의 개수를 줄여서 오버피팅을 막는 행위  

>예)  
특성으로 pH, 당도, 알콜수 이렇게 3가지가 있는데 여기서 $\sqrt{특성개수}$ 만큼.  
=> pH, 당도 만 랜덤으로 뽑음

이렇게 하는 이유는 결정트리의 성능이 너무 좋아 오버피팅되기 때문에  
랜덤으로 값을 뽑아내서 고의적으로 성능을 끌어내리는 것이다.

이렇게 만들어진 트리를 여러개 만들어서 일반화 하면 랜덤 포레스트가 만들어진다.  
(기본적으로 100개의 트리 생성)

In [1]:
## 랜덤포레스트

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

wine = pd.read_csv('https://bit.ly/wine_csv_data')

data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)

In [2]:
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier

# n_jobs가 뭐지???
rf = RandomForestClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(rf, train_input, train_target, return_train_score=True, n_jobs=-1)

# 이름은 테스트 스코어지만 train만 넣어주었기 때문에 사실 검증세트다.
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9973541965122431 0.8905151032797809


In [3]:
rf.fit(train_input, train_target)
# 데이터의 중요도 출력
print(rf.feature_importances_)

[0.23167441 0.50039841 0.26792718]


In [4]:
# oob(Out Of Bagging) : 부트스트랩 샘플을 뽑아낼 때 뽑아내지 않은, 트리를 만들지 않은 샘플들
# oob는 트리에 사용되지 않으므로 검증세트로 사용할 수 있음
rf = RandomForestClassifier(oob_score=True, n_jobs=-1, random_state=42)

rf.fit(train_input, train_target)
print(rf.oob_score_)

0.8934000384837406


### 엑스트라트리
랜덤포레스트와 비슷한데 부트스트랩 샘플을 사용하지 않음  
무작위성을 다른 곳에서 찾은 알고리즘 이라고 생각하면 댐  
노트 분할을 무작위로 한다(추가 설명이 필요할 것 같다...)  
가장 좋은 불순도 차이를 나타내는 것을 찾아냄  

In [5]:
## 엑스트라트리

# ExtraTree"s"Classifier이다.
# 부트스트랩 샘플을 사용하지 않는다
# 부트스트랩 샘플을 사용하지 않고 overfitting을 방지하는 방법은?
# 특성중에서 랜덤하게 뽑아낸 다음에 불순도가 가장 높은 것을 고른다
from sklearn.ensemble import ExtraTreesClassifier

et = ExtraTreesClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(et, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

et.fit(train_input, train_target)
print(et.feature_importances_)

0.9974503966084433 0.8887848893166506
[0.20183568 0.52242907 0.27573525]


### 그레이디언트 부스팅
트리 앙상블 중에 가장 성능이 좋은 알고리즘이다.
성능이 너무 좋아서 구현한 패키지들이 많다.  
LisghtGBM, CatBoost, XGBoost 등  

경사하강법에서의 경사와 유사하게 손실함수로 최저점을 찾는 방식을 사용한다.
손신함수로는
- 회귀 -> MSE 사용
- 분류 -> 로지스틱 사용

In [6]:
## 그레이디언트 부스팅

from sklearn.ensemble import GradientBoostingClassifier

gb = GradientBoostingClassifier(random_state=42)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.8881086892152563 0.8720430147331015


In [7]:
# n_estimators : 병렬로 생성되는 트리의 개수
# learning_rate : 
gb = GradientBoostingClassifier(n_estimators=500, learning_rate=0.2, random_state=42)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9464595437171814 0.8780082549788999


In [8]:
gb.fit(train_input, train_target)
print(gb.feature_importances_)

[0.15872278 0.68010884 0.16116839]


### 히스토그램 기반 부스팅
Histogram - 데이터를 일정 구간으로 해서 막대그래프를 그림  
훈련 데이터를 256개 구간으로 분할해서 특성 변환  
예전엔 실험적 상태여서 명시적으로 enable를 해줘야하는데 지금은 정식 버전으로 나왔다.

In [9]:
## 히스토그램 기반 부스팅

# 사이킷런 1.0 버전 아래에서는 다음 라인의 주석을 해제하고 실행하세요.
# from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier

hgb = HistGradientBoostingClassifier(random_state=42)
scores = cross_validate(hgb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9321723946453317 0.8801241948619236


#### permutation_importance
X값(feature)이 여러개가 있는데 하나의 X값 순서를 막 섞고 모델을 만들어봤는데  
예측값이 완전히 틀어졌다 -> 이 X값이 Y값에 영향을 많이 미친다는 것을 알 수 있다.  
이것으로 각 feature당 중요도를 알 수가 있다.  

In [10]:
from sklearn.inspection import permutation_importance

hgb.fit(train_input, train_target)
result = permutation_importance(hgb, train_input, train_target, n_repeats=10,
                                random_state=42, n_jobs=-1)
print(result.importances_mean)

# 당도를 섞어봤는데 0.2만큼의 정확도가 떨어졌다.  
# => 당도의 중요도가 크다는 뜻이다.

[0.08876275 0.23438522 0.08027708]


In [11]:
result = permutation_importance(hgb, test_input, test_target, n_repeats=10,
                                random_state=42, n_jobs=-1)
print(result.importances_mean)

# Test 데이터로도 측정해봤는데 이때도 당도가 0.2만큼의 정확도가 떨어졌다  
# => 실전에서도 당도는 중요도가 크다는 것을  확인할 수 있다.

[0.05969231 0.20238462 0.049     ]


In [12]:
hgb.score(test_input, test_target)

0.8723076923076923

### XGBoost & LightGBM
사이킷런에서 분리되어서 아예 다른 패키지로 나와버린 그레이디언트 부스트 알고리즘

In [13]:
#### XGBoost

from xgboost import XGBClassifier

xgb = XGBClassifier(tree_method='hist', random_state=42)
scores = cross_validate(xgb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

#### LightGBM

from lightgbm import LGBMClassifier

lgb = LGBMClassifier(random_state=42)
scores = cross_validate(lgb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

/home/solstice/anaconda3/lib/python3.9/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index
/home/solstice/anaconda3/lib/python3.9/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index
/home/solstice/anaconda3/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserW

0.9555033709953124 0.8799326275264677
0.935828414851749 0.8801251203079884
